In [16]:
from bs4 import BeautifulSoup as bs
import requests
import re

In [24]:
def remove_reference(text):
    return re.sub(r"\[\d+\]", "", text).strip()

def get_content_value(row_data):
    if row_data.find("li"):
        return [remove_reference(li.get_text(" ", strip=True).replace("\xa0", " ")) for li in row_data.find_all("li")]

    
    else:
        return remove_reference(row_data.get_text(" ", strip=True).replace("\xa0", " "))

def get_info_box(url):

    r = requests.get(url)

    soup = bs(r.content)
    contents = soup.prettify()
    info_box = soup.find(class_="infobox vevent")
    info_rows = info_box.find_all("tr")

    movie_info = {}
    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['title'] = row.find('th').get_text(" ", strip=True)
        elif index == 1:
            continue
        else:
            content_key = row.find('th').get_text(' ', strip=True)
            content_value = get_content_value(row.find('td'))
            movie_info[content_key] = content_value

    return movie_info

In [25]:
get_info_box("https://en.wikipedia.org/wiki/Cinderella_(1950_film)")

{'title': 'Cinderella',
 'Directed by': ['Wilfred Jackson', 'Hamilton Luske', 'Clyde Geronimi'],
 'Story by': ['William Peet',
  'Ted Sears',
  'Homer Brightman',
  'Kenneth Anderson',
  'Erdman Penner',
  'Winston Hibler',
  'Harry Reeves',
  'Joe Rinaldi'],
 'Based on': '" Cinderella " by Charles Perrault',
 'Produced by': ['Walt Disney', 'Ben Sharpsteen'],
 'Starring': ['Ilene Woods',
  'Eleanor Audley',
  'Verna Felton',
  'Rhoda Williams',
  'James MacDonald',
  'Luis van Rooten'],
 'Edited by': 'Donald Halliday',
 'Music by': ['Oliver Wallace', 'Paul Smith'],
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'RKO Radio Pictures',
 'Release dates': ['February 15, 1950 ( 1950-02-15 ) (Boston)',
  'March 4, 1950 ( 1950-03-04 ) (United States)'],
 'Running time': '74 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$2.2 million',
 'Box office': '$182 million'}

In [26]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")

# Convert to beautiful soup object
soup = bs(r.content)

movies = soup.select(".wikitable.sortable i a")

base_path = "https://en.wikipedia.org/"
movie_info_list = []

for index, movie in enumerate(movies):
    if index == 5:
        break 
    
    if index % 10 == 0:
        print(index)
    try:
        relative_path = movie['href']
        full_path = base_path + relative_path
        title = movie['title']

        movie_info_list.append(get_info_box(full_path))
    
    except Exception as e:
        print(movie.get_text())
        print(e)


0


In [27]:
movie_info_list

[{'title': 'Snow White and the Seven Dwarfs',
  'Directed by': ['David Hand',
   'Perce Pearce',
   'William Cottrell',
   'Larry Morey',
   'Wilfred Jackson',
   'Ben Sharpsteen'],
  'Story by': ['Ted Sears',
   'Richard Creedon',
   'Otto Englander',
   'Dick Rickard',
   'Earl Hurd',
   'Merrill De Maris',
   'Dorothy Ann Blank',
   'Webb Smith'],
  'Based on': '" Snow White " by the Brothers Grimm',
  'Produced by': 'Walt Disney',
  'Starring': ['Adriana Caselotti',
   'Roy Atwell',
   'Pinto Colvig',
   'Otis Harlan',
   'Scotty Mattraw',
   'Billy Gilbert',
   'Eddie Collins'],
  'Music by': ['Frank Churchill', 'Leigh Harline', 'Paul Smith'],
  'Production company': 'Walt Disney Productions',
  'Distributed by': 'RKO Radio Pictures',
  'Release dates': ['December 21, 1937 ( 1937-12-21 ) ( Carthay Circle Theatre )',
   'February 4, 1938 ( 1938-02-04 ) (United States)'],
  'Running time': '83 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$1.5 million

In [8]:
import re

number = r"\d+(,\d{3})*\.*\d*"

print(re.search(number, '$ Million'))

None


In [9]:
date = 'September 6, 2010 ( El Cardio)'
date.split("(")[0].strip()

'September 6, 2010'

In [11]:
import requests

api_key = "http://www.omdbapi.com/?apikey=[yourkey]&"

base_url = "https://www.omdbapi.com/"

params = {
    'apikey': api_key,
    's': 'Into the Woods'
}

response = requests.get(base_url, params=params)
if response.status_code == 200:
    a = response.json

else:
    print(f"Fail to retrieve data {response.status_code}")

Fail to retrieve data 401


In [ ]:
for movie in movie_info_list:
    title = movie['title']
    omdb_info = get_omdb_info(title)
    movie['imdb'] = get_omdb_info.get('imdbRating', None)
    movie['metascore'] = get_omdb_info('Metascore', None)
    movie['rotten_tomatoes'] = get_rotten_tomato_score(omdb_info)